In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

data = pd.read_csv("DATA/Robust_Scaler_result.csv",encoding="cp949")
data

,CUSTNO,GNO,CBSCORE,CBSCOREGRD,CREDITOTAMT,YSALEAMT,ESTMM,ASSETAMT,IMSAAMT,IMJUAMT,...,환율,GDP,소상공인체감지수,실업률,물가지수,국고채,금리,유가등락률,소비자심리지수,ONEHOT
0,475821,l180202101898,1.015873,1,0.230769,10.743169,0.014085,10,0.0,0.0,...,0.009592,1.885958,51.40,0.000000,1.724868,-1.256410,-4.0,1.750600,0.798246,3
1,74417,l230201700120,0.206349,2,0.205128,6.027322,-0.380282,25,-0.5,25.0,...,0.537170,-0.901805,53.80,0.666667,-0.640212,-0.307692,-1.0,1.100719,-0.596491,8
2,387787,l110201603233,0.396825,1,-0.282051,3.775956,2.098592,10,0.0,0.0,...,0.978417,-1.182547,72.60,0.000000,-1.417989,-0.461538,-1.0,0.110312,-0.482456,7
3,395418,l230201700254,0.000000,2,0.076923,3.562842,-0.352113,10,0.0,0.0,...,0.520384,-0.901805,76.20,-0.333333,-0.677249,-0.205128,-1.0,0.429257,-0.394737,3
4,190372,l200201601418,-2.476190,6,-0.102564,1.437158,1.845070,80,-0.4,0.0,...,0.446043,-1.357134,63.20,-0.333333,-1.867725,-1.461538,-1.0,-0.618705,-0.017544,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39112,514919,l240201902522,-0.253968,3,-0.282051,-0.393443,-0.338028,0,-0.5,0.0,...,0.772182,1.448316,65.90,0.000000,0.523810,-1.025641,-1.0,0.489209,0.008772,7
39113,357361,l160201601877,-1.603175,5,0.000000,-0.393443,-0.366197,10,-0.5,10.0,...,1.122302,-1.499675,64.60,-0.333333,-1.767196,-1.179487,-1.0,-0.786571,-0.210526,16
39114,340606,l270201900572,-2.349206,6,-0.051282,-0.393443,0.140845,0,-0.5,0.0,...,-0.028777,0.593436,67.85,0.333333,0.190476,0.000000,1.0,-0.515588,-0.061404,19
39115,505207,l190201902523,1.063492,1,-0.282051,-0.393443,-0.436620,0,-0.5,0.0,...,0.733813,1.448316,72.80,0.000000,0.391534,-0.769231,-1.0,-0.139089,0.043860,16


In [4]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUSAAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '소상공인체감지수', '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','ONEHOT']]
act['ACTCD'] = data['ACTCD']
       

In [5]:
from sklearn.model_selection import train_test_split

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 22) (11736, 22) (27381,) (11736,)


In [6]:
from imblearn.over_sampling import BorderlineSMOTE, ADASYN
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
from imblearn.combine import SMOTEENN, SMOTETomek

#언더 샘플링
enn = EditedNearestNeighbours(kind_sel="all", n_neighbors=10)
tomekl = TomekLinks()

#오버 샘플링
bsmote = BorderlineSMOTE(random_state=42)
adasyn = ADASYN(random_state=42)

#혼합 샘플링
smotee = SMOTEENN(random_state=42)
smoteT = SMOTETomek(random_state=42)


X_under1_train, Y_under1_train = enn.fit_resample(train_x, train_y)
X_under2_train, Y_under2_train = tomekl.fit_resample(train_x, train_y)

X_over1_train, Y_over1_train = bsmote.fit_resample(train_x,train_y)
X_over2_train, Y_over2_train = adasyn.fit_resample(train_x,train_y)

X_comb1_train, Y_comb1_train = smotee.fit_resample(train_x, train_y)
X_comb2_train, Y_comb2_train = smoteT.fit_resample(train_x, train_y)

In [7]:
def cut_off(y, threshold) :
    Y = y.copy()  # 대문자 Y를 새로운 변수로 하여 기존의 y값에 영향이 가지 않도록 한다.
    Y[Y>threshold] = 1
    Y[Y<threshold] = 0
    return Y.astype(int)

from sklearn.metrics import confusion_matrix
# confusion matrix accuracy(정확도) 계산함수

def acc(cfmat):
    return (cfmat[0,0] + cfmat[1,1])/(cfmat[0,0] + cfmat[1,1] + cfmat[0,1] + cfmat[1,0])

# 오버샘플링

1. BorderlineSMOTE

In [8]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [9]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)

In [10]:
from sklearn.model_selection import cross_val_score
cross_val_score(eclf, X_over1_train, Y_over1_train, cv=5).mean()

0.8403857745629898

In [11]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier()

DecisionTreeClassifier()

In [12]:
AdaBoostClassifier()

AdaBoostClassifier()

In [13]:
params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

In [14]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_over1_train, Y_over1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.55,
 'n_estimators': 25}

In [13]:
grid.best_estimator_.feature_importances_

array([0.07766098, 0.04684005, 0.02373662, 0.08715234, 0.07228255,
       0.07062534, 0.01287007, 0.0402632 , 0.08371823, 0.13837393,
       0.11922424, 0.        , 0.00126807, 0.04223189, 0.00599657,
       0.00151928, 0.02532738, 0.02047163, 0.0243234 , 0.        ,
       0.00419939, 0.10191486])

In [15]:
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score

dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.55, n_estimators = 26)
model = abc.fit(X_over1_train, Y_over1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [16]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[8051, 2615],
       [ 476,  594]], dtype=int64)

In [17]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.94      0.75      0.84     10666
        정상보증       0.19      0.56      0.28      1070

    accuracy                           0.74     11736
   macro avg       0.56      0.65      0.56     11736
weighted avg       0.87      0.74      0.79     11736



In [16]:
accuracy = acc(cfmat)
accuracy

0.7547716428084527

In [17]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.1814658210007047
recall:  0.48130841121495327
f1 score:  0.26356192425793246
accuracy:  0.6317567745180335


2. ADASYN

In [18]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_over2_train, Y_over2_train, cv=5).mean()

0.8688921613678321

In [18]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_over2_train, Y_over2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.45,
 'n_estimators': 27}

In [20]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.45, n_estimators = 27)
model = abc.fit(X_over2_train, Y_over2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [21]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[7949, 2717],
       [ 521,  549]], dtype=int64)

In [23]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.94      0.75      0.83     10666
        정상보증       0.17      0.51      0.25      1070

    accuracy                           0.72     11736
   macro avg       0.55      0.63      0.54     11736
weighted avg       0.87      0.72      0.78     11736



In [22]:
accuracy = acc(cfmat)
accuracy

0.7240967961826857

In [23]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.16275764036958068
recall:  0.4280373831775701
f1 score:  0.23583934088568487
accuracy:  0.6035742888136116


# 언더샘플링

1. EditedNearestNeighbours

In [24]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_under1_train, Y_under1_train, cv=5).mean()

0.8741021539003103

In [24]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_under1_train, Y_under1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 7,
 'learning_rate': 0.6,
 'n_estimators': 27}

In [25]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 7)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 27)
model = abc.fit(X_under1_train, Y_under1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [26]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[9903,  763],
       [ 774,  296]], dtype=int64)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.93      0.93      0.93     10666
        정상보증       0.28      0.28      0.28      1070

    accuracy                           0.87     11736
   macro avg       0.60      0.60      0.60     11736
weighted avg       0.87      0.87      0.87     11736



In [28]:
accuracy = acc(cfmat)
accuracy

0.8611111111111112

In [29]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.2719869706840391
recall:  0.3121495327102804
f1 score:  0.2906875543951262
accuracy:  0.6141658970508086


2. TomekLinks

In [30]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_under2_train, Y_under2_train, cv=5).mean()

0.9063297857647399

In [28]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_under2_train, Y_under2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 7,
 'learning_rate': 0.55,
 'n_estimators': 25}

In [29]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 7)
abc = AdaBoostClassifier(learning_rate = 0.55, n_estimators = 25)
model = abc.fit(X_under2_train, Y_under2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [30]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[10666,     0],
       [ 1070,     0]], dtype=int64)

In [31]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.91      1.00      0.95     10666
        정상보증       0.00      0.00      0.00      1070

    accuracy                           0.91     11736
   macro avg       0.45      0.50      0.48     11736
weighted avg       0.83      0.91      0.87     11736



C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
accuracy = acc(cfmat)
accuracy

0.9088275391956373

In [35]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.0
recall:  0.0
f1 score:  0.0
accuracy:  0.5


C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 혼합샘플링

1. SMOTEENN

In [36]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_comb1_train, Y_comb1_train, cv=5).mean()

0.875260659094

In [32]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_comb1_train, Y_comb1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 24}

In [33]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 24)
model = abc.fit(X_comb1_train, Y_comb1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [34]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[6667, 3999],
       [ 314,  756]], dtype=int64)

In [35]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.96      0.63      0.76     10666
        정상보증       0.16      0.71      0.26      1070

    accuracy                           0.63     11736
   macro avg       0.56      0.67      0.51     11736
weighted avg       0.88      0.63      0.71     11736



In [40]:
accuracy = acc(cfmat)
accuracy

0.6411042944785276

In [41]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.1551799824407375
recall:  0.6607476635514019
f1 score:  0.2513330963384287
accuracy:  0.6499406797036965


2. SMOTETomek

In [46]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_comb2_train, Y_comb2_train, cv=5).mean()

0.863866503038396

In [36]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_comb2_train, Y_comb2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.55,
 'n_estimators': 27}

In [37]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.55, n_estimators = 27)
model = abc.fit(X_comb2_train, Y_comb2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [38]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[7961, 2705],
       [ 503,  567]], dtype=int64)

In [39]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.94      0.75      0.83     10666
        정상보증       0.17      0.53      0.26      1070

    accuracy                           0.73     11736
   macro avg       0.56      0.64      0.55     11736
weighted avg       0.87      0.73      0.78     11736



In [101]:
accuracy = acc(cfmat)
accuracy

0.7366223585548739

In [102]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.1694471704285247
recall:  0.48411214953271026
f1 score:  0.2510298037315241
accuracy:  0.623033010824859
